In [1]:
import pandas as pd
import numpy as np
import csv
import string
import matplotlib.pyplot as plt
from matplotlib import style
import datetime

#read full csv including review text

infile = 'review_full.csv'
df = pd.read_csv(infile,skipinitialspace = True)
df.columns = ['ProductId','Helpful','Rating','ReviewSummary','ReviewDate','ReviewerID','Reviewer']

#read  csv without review text

#infile = 'review.csv'
#df = pd.read_csv(infile,skipinitialspace = True)
#df.columns = ['ProductId','Helpful','Rating','ReviewDate','Reviewer']


print(df.info())

#clear NaN Rows
df = df[df['ProductId'] != 'NaN']

#get the helpful score to put into 2 columns
s = df['Helpful'].str.split(',', expand=True).astype(str)
df['Helpfulness'] = s[0].str.strip('[')
df['helpfulness_denominator'] = s[1].str.strip(']')
#convert to numberic
df['Helpfulness'] = df['Helpfulness'].convert_objects(convert_numeric=True)
df['helpfulness_denominator'] = df['helpfulness_denominator'].convert_objects(convert_numeric=True)

#df['helpfulness_denominator'] = df['helpfulness_denominator'].convert_objects(convert_numeric=True)
df['helpfulscore'] = np.where(df['helpfulness_denominator']>0, df['Helpfulness']/df['helpfulness_denominator'],0)
df['helpfulscore'] = df['Helpfulness']/df['helpfulness_denominator']

#Get the length of review summary

df['reviewtextcount'] = df['ReviewSummary'].str.len()


s = df['ReviewDate'].str.split(',', expand=True).astype(str)
df['ReviewDate_year'] = s[1]
s2 = s[0].str.split(' ', expand=True).astype(str)
df['ReviewDate_month'] = s2[0]
df['ReviewDate_date'] = s2[1]
df['ReviewDate_formatted']=pd.to_datetime(df['ReviewDate_year'].str.strip()+df['ReviewDate_month'].str.strip()+df['ReviewDate_date'].str.strip(), format='%Y%m%d')

# get the days of the week
df['day_of_week'] = df['ReviewDate_formatted'].dt.dayofweek
#days = {0:'Mon',1:'Tues',2:'Weds',3:'Thur',4:'Fri',5:'Sat',6:'Sun'}
#df['day_of_week'] = df['day_of_week'].apply(lambda x: days[x])


#####      read the meta data  ############
metafile = 'meta.csv'
df_meta = pd.read_csv(metafile,skipinitialspace = True)
df_meta.columns = ['ProductId','title','price']

#clear NaN Rows
df_meta = df_meta[df_meta['price']>0]
#print(df_meta.head())

#####         Merge with Meta data to get price infor ###################

df_full = pd.merge(df,df_meta,on= 'ProductId')

#####       some basics about the merged data     #######
print(df_full.describe())

print(df_full.corr(method='pearson'))




<class 'pandas.core.frame.DataFrame'>
Int64Index: 7824545 entries, 0 to 7824544
Data columns (total 7 columns):
ProductId        object
Helpful          object
Rating           float64
ReviewSummary    object
ReviewDate       object
ReviewerID       object
Reviewer         object
dtypes: float64(1), object(6)
memory usage: 477.6+ MB
None
               Rating     Helpfulness  helpfulness_denominator  \
count  7351816.000000  7351816.000000           7351816.000000   
mean         4.022256        1.969309                 2.520309   
std          1.375335       23.336054                24.536412   
min          1.000000        0.000000                 0.000000   
25%          3.000000        0.000000                 0.000000   
50%          5.000000        0.000000                 0.000000   
75%          5.000000        1.000000                 1.000000   
max          5.000000    30735.000000             31453.000000   

         helpfulscore  reviewtextcount     day_of_week           

In [ ]:
##  1. Agg by month and plot by year, count number of comments, and get the mean of helpfulness score
inputyear= input("Pls select inpput year: ")
df_full_yearly = df_full[df_full['ReviewDate_year'].str.strip() == inputyear]
print(df_full_yearly.head(5))

df_monthlycount = df_full_yearly[['ProductId','Reviewer','ReviewDate_month']].groupby(['ReviewDate_month']).count()
#print(df_monthlycount)

df_helful_monthlymean =  df_full_yearly[['helpfulscore','ReviewDate_month']].groupby(['ReviewDate_month']).agg('mean')
#print(df_helful_monthlymean.head(12))



#####Plot 1 #################
print(type(df_monthlycount))

import matplotlib.pyplot as plt
df_monthlycount.to_csv('df_monthlycount.csv')
df = pd.read_csv('df_monthlycount.csv')
df.plot(x ='ReviewDate_month',y=['ProductId', 'Reviewer'],kind='line',title = 'Counts of review by Months')
plt.show()

In [ ]:
##  2. Agg by days of week ################
inputyear= input("Pls input year(2012/2013/2014): ")
df_full_yearly = df_full[df_full['ReviewDate_year'].str.strip() == inputyear]print(df_full_yearly.head())
df_full_daysofweek = df_full_yearly[['ProductId','Reviewer','day_of_week']].groupby(['day_of_week']).count()

#####Plot 2 #################
print(type(df_full_daysofweek))

import matplotlib.pyplot as plt
df_full_daysofweek.to_csv('df_daysofweekcount.csv')
df = pd.read_csv('df_daysofweekcount.csv')
df.plot(x ='day_of_week',y=['ProductId', 'Reviewer'],kind='line',title = 'Counts day of week')
plt.show()

In [ ]:
# price and rating score correlation  WIP

#inputyear= input("Pls input year(2012/2013/2014): ") 
inputyear = '2013'

df_full_yearly = df_full[df_full['ReviewDate_year'].str.strip() == inputyear]

df_review_count = df_full_yearly[['ProductId','Reviewer']].groupby(['ProductId']).count()

print(df_review_count.head())
#df_new.columns = ['ProductId','Helpful','Rating','ReviewDate','Reviewer','count_by_prodcut']
df = pd.merge(df_full_yearly,df_review_count,on= 'ProductId')

print(df.head())
#df_full_yearly.corr(method='pearson', min_periods=1)

In [ ]:
#Analyize the length and, review score and helpfulness total relationship, plot into scatter 

inputyear = '2013'

df_full_2 = df_full[df_full['ReviewDate_year'].str.strip() == inputyear]

plt.scatter(df_full_2['Rating'],df_full_2['reviewtextcount'],s=df_full_2['Helpfulness'])
plt.show()


In [ ]:
#Analyize the length and, review score and price total relationship, plot into scatter 

inputyear = '2013'
df_full_yearly = df_full[df_full['ReviewDate_year'].str.strip() == inputyear]

plt.scatter(df_full_yearly['Rating'],df_full_yearly['reviewtextcount'],s=df_full_yearly['price'])
plt.show()



In [ ]:
#Analysis the trend of review length in the days of week
inputyear = '2013'
df_full_yearly = df_full[df_full['ReviewDate_year'].str.strip() == inputyear]

df_avgreviewlengh_daysofweek = df_full_yearly[['day_of_week','reviewtextcount']].groupby(['day_of_week']).mean()

import matplotlib.pyplot as plt
df_avgreviewlengh_daysofweek.to_csv('df_avgreviewlengh_daysofweek.csv')
df_avgreviewlengh_daysofweek = pd.read_csv('df_avgreviewlengh_daysofweek.csv')

df_avgreviewlengh_daysofweek.plot(x ='day_of_week',y=['reviewtextcount'],kind='line')
plt.show()

In [ ]:
#Analyize rating versus helpfuness

inputyear = '2013'

df_full_yearly = df_full[df_full['ReviewDate_year'].str.strip() == inputyear]
plt.scatter(df_full_yearly['Rating'],df_full_yearly['Helpfulness'])
plt.show()




In [ ]:
#Analyize rating count versus price

inputyear = '2013'
df_full_yearly = df_full[df_full['ReviewDate_year'].str.strip() == inputyear]



In [19]:

df_reviewcount_pricerange = df_full_yearly[['price','Reviewer']].groupby(['price']).count()

import matplotlib.pyplot as plt

df_reviewcount_pricerange.to_csv('df_reviewcount_pricerange.csv')
df = pd.read_csv('df_reviewcount_pricerange.csv')
df.plot(x ='price',y=['Reviewer'],kind='line',title = 'Review by price range')
plt.show()



C:\Anaconda3\lib\site-packages\ipykernel\__main__.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [26]:

#Analyize price range versus helpfulness

inputyear = '2013'

df_full_yearly = df_full[df_full['ReviewDate_year'].str.strip() == inputyear]
df_full_yearly['pricerange'] = np.around(df_full_yearly['price']/10,0)

df_helpful_pricerange = df_full_yearly[['pricerange','Helpfulness']].groupby(['pricerange']).mean()



df_helpful_pricerange.to_csv('df_helpful_pricerange.csv')
df = pd.read_csv('df_helpful_pricerange.csv')

df.plot(x ='pricerange',y=['Helpfulness'],kind='line',title = 'Helpfulness by price range')
plt.show()


C:\Anaconda3\lib\site-packages\ipykernel\__main__.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [6]:
#Addtional - to count unique number of revies and prodcuts

df_2=df_full['ProductId'].value_counts(normalize=True,dropna=True)
print("Number of Unique Product: ")
print(df_2.count())

df_3=df_full['ReviewerID'].value_counts(normalize=True,dropna=True)
print('Number of Unique Reviewer: ')
print(df_3.count())



Number of Unique Product: 
370402
Number of Unique Reviewer: 
4009194


In [2]:
#Addtional - to get the number of helpfulness score >0 of reviews

df_4 = df_full[df_full['helpfulscore']>0.5]

print(df_4.count())



ProductId                  2104471
Helpful                    2104471
Rating                     2104471
ReviewSummary              2101954
ReviewDate                 2104471
ReviewerID                 2104471
Reviewer                   2095234
Helpfulness                2104471
helpfulness_denominator    2104471
helpfulscore               2104471
reviewtextcount            2101954
ReviewDate_year            2104471
ReviewDate_month           2104471
ReviewDate_date            2104471
ReviewDate_formatted       2104471
day_of_week                2104471
title                      2042638
price                      2104471
dtype: int64


inputyear= input("Pls input year(2012/2013/2014): ")
df_full_yearly = df_full[df_full['ReviewDate_year'].str.strip() == inputyear]

df_price_rating_product = df_full_yearly[['price','Rating','ProductId']].groupby(['ProductId']).agg('mean')
print(df_price_rating_product.head())